<H1 style="background-color:LightGreen;">
    <center>
         Kaggle
     </center>
 </H1>

https://www.kaggle.com/competitions/spooky-author-identification/overview

Topic modeling


<H1 style="background-color:LightGreen;">
    <center>
         NLP Notes:
     </center>
 </H1>

<li> Stemming identifies the common root form of a word by removing or replacing word suffixes (e.g. “flooding” is stemmed as “flood”) 
<li> Lemmatization identifies the inflected forms of a word and returns its base form (e.g. “better” is lemmatized as “good”).

https://towardsdatascience.com/text-classification-with-bert-in-pytorch-887965e5820f

<H1 style="background-color:LightGreen;">
    <center>
         Imports
     </center>
 </H1>

In [1]:
#-- Auto reload:
%load_ext autoreload
%autoreload 2

In [2]:
from matplotlib                          import pyplot
from matplotlib.image                    import imread
from torchvision.datasets                import ImageFolder
from torch.optim.lr_scheduler            import OneCycleLR
from sklearn.metrics                     import r2_score
from sklearn.feature_extraction.text     import TfidfVectorizer
from nltk.stem.porter                    import PorterStemmer
from nltk.corpus                         import stopwords
from nltk.tokenize                       import sent_tokenize, word_tokenize
from nltk.stem                           import WordNetLemmatizer
from transformers                        import BertTokenizer
from transformers                        import BertModel

import numpy                  as np
import matplotlib.pyplot      as plt
import matplotlib.image       as mpimg
import torchvision.transforms as transforms
import torch.nn               as nn
import torch.nn.functional    as F
import torch.optim            as optim
import pandas                 as pd


import torchinfo
import torch
import os
import random
import matplotlib
import torchvision
import torchinfo
import gc 
import os
import time
import nltk
import string


<H1 style="background-color:LightGreen;">
    <center>
         Load Data
     </center>
 </H1>

In [3]:
TRAIN_FOLDER = "/home/amitli/Datasets/spooky-author-identification/spooky-author-identification/train.csv"
TEST_FOLDER  = "/home/amitli/Datasets/spooky-author-identification/spooky-author-identification/test.csv"

df_train = pd.read_csv(TRAIN_FOLDER)
df_train.drop(['id'], axis=1, inplace=True)
df_train.head()

,text,author
0,"This process, however, afforded me no means of...",EAP
1,It never once occurred to me that the fumbling...,HPL
2,"In his left hand was a gold snuff box, from wh...",EAP
3,How lovely is spring As we looked from Windsor...,MWS
4,"Finding nothing else, not even gold, the Super...",HPL


<H1 style="background-color:LightGreen;">
    <center>
         EDA
     </center>
 </H1>

In [ ]:
df_train.groupby(by='author').count()

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

<H1 style="background-color:LightYellow;">
    <center>
         TOP tfidf (10 words) of each author
     </center>
 </H1>

In [ ]:


def Stemmer(sentences):
    stemmer = PorterStemmer()
    
    for i in range(len(sentences)):
        words = word_tokenize(sentences[i])    
        words = [stemmer.stem(word) for word in words if word not in set(stopwords.words('english'))]
        sentences[i]=' '.join(words)
    return sentences

def Lemmmatizer(sentences):
    lemmmatizer = WordNetLemmatizer()

    for i in range(len(sentences)):
        words = word_tokenize(sentences[i])  
        words = [lemmmatizer.lemmatize(word.lower()) for word in words if word not in set(stopwords.words('english'))]
        sentences[i]=' '.join(words)   
    
    return sentences

In [ ]:
vectorizer = TfidfVectorizer(stop_words="english", max_features=10)
authors    = df_train.groupby(by='author')

for author in authors:
    df_tmp = author[1].reset_index()
    res = Lemmmatizer(df_tmp['text'].values)
    res = Stemmer(res)
    X   = vectorizer.fit_transform(res)
    print(f"Author: {author[0]}, Top words: {vectorizer.get_feature_names_out()}")
   


<H1 style="background-color:LightYellow;">
    <center>
         BERT
     </center>
 </H1>

<img src="../bert.png">

<H1 style="background-color:LightYellow;">
    <center>
         Create dataset
     </center>
 </H1>

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [5]:

labels = {'EAP':0,
          'HPL':1,
          'MWS':2        
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        # 512 - maximum size of tokens that can be fed into BERT model.
        
        # bert_input = tokenizer()
        #       bert_input['input_ids']         - representation of each token
        #       bert_input['token_type_ids']    - dentifies in which sequence a token belongs [0/1]
        #       bert_input['attention_mask']    - binary mask that identifies whether a token is a real word 
        #                                         or just padding. 
        #                                         If the token contains [CLS], [SEP], or any real word
        
        self.labels = [labels[label] for label in df['author']]
        self.texts  = [tokenizer(text, 
                                padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y     = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [6]:
train            = Dataset(df_train)
train_dataloader = torch.utils.data.DataLoader(train, batch_size=4, shuffle=True)

In [7]:
x, y = next(iter(train_dataloader))

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
cpu_device = "cpu"

mask     = x['attention_mask'].to(cpu_device)
input_id = x['input_ids'].squeeze(1).to(cpu_device)
                

In [19]:
input_id.shape, mask.shape, len(y)

(torch.Size([4, 512]), torch.Size([4, 1, 512]), 4)

<H1 style="background-color:LightYellow;">
    <center>
         Create Model 
     </center>
 </H1>

In [24]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert    = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear  = nn.Linear(768, 3)
        self.relu    = nn.ReLU()

    def forward(self, input_id, mask):
        
        #
        # pooled_output - embedding vector of [CLS] token
        #
        vEmbeddingToken, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output                 = self.dropout(pooled_output)
        linear_output                  = self.linear(dropout_output)
        final_layer                    = self.relu(linear_output)

        return final_layer

In [25]:
torchinfo.summary(BertClassifier(), ((4, 512),(4, 1, 512)))

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: []

<H1 style="background-color:LightYellow;">
    <center>
         Train Model 
     </center>
 </H1>

In [ ]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)
                
                
                print(f"type(input_id) = {type(input_id)}, type(mask) = {type(mask)}")
                print(f"shape(input_id) = {input_id.shape}, shape(mask) = {mask.shape}")
                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  


In [ ]:
from torch.optim import Adam
EPOCHS = 5
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_train, LR, EPOCHS)